In [ ]:
import marimo as mo
import polars as pl

# HathiTrust page-level genre predictions and PPA

This analysis is based on:

> Underwood, Ted (2014). Page-Level Genre Metadata for English-Language Volumes in HathiTrust, 1700-1922. figshare. Dataset. https://doi.org/10.6084/m9.figshare.1279201.v1

To replicate this analysis, download the full zip file and expand it in the `data/` directory.

## Identify overlap between page-level genre volumes and PPA

First, we need to find which PPA volumes are included in this dataset.

In [ ]:
# load PPA work-level metadata and limit to HathiTrust content
ppa_ht_df = pl.read_csv("data/ppa/ppa_work_metadata.csv").filter(
    pl.col("ppa_source").eq("HathiTrust")
)
ppa_ht_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"ppa_work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"ppa_source_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"ppa_cluster_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"ppa_work_title\":\"Treatise on French pronunciation and genders\",\"ppa_work_author\":\"Lafont, J. B. Angelvy\",\"ppa_work_year\":1864,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t1hh7pc2q\",\"ppa_source_id\":\"aeu.ark:/13960/t1hh7pc2q\",\"ppa_cluster_id\":\"aeu.ark:/13960/t1hh7pc2q\",\"ppa_work_title\":\"An essay on the elements, accents, & prosody, of the English language\",\"ppa_work_author\":\"Odell, Jonathan, 1737-1818\",\"ppa_work_year\":1805,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary', 'Typographically Unique']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t1pg22p71-p126\",\"ppa_source_id\":\"aeu.ark:/13960/t1pg22p71\",\"ppa_cluster_id\":\"aeu.ark:/13960/t1pg22p71-p126\",\"ppa_work_title\":\"On the Galliambic Metre\",\"ppa_work_author\":\"Allen, [Charles] Grant [Blairfindie].\",\"ppa_work_year\":1892,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary', 'Original Bibliography', 'Typographically Unique']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t3514pf3k\",\"ppa_source_id\":\"aeu.ark:/13960/t3514pf3k\",\"ppa_cluster_id\":\"aeu.ark:/13960/t3514pf3k\",\"ppa_work_title\":\"An elementary Greek grammar\",\"ppa_work_author\":\"Goodwin, William W. (William Watson), 1831-1912\",\"ppa_work_year\":1879,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t3jw96h3m\",\"ppa_source_id\":\"aeu.ark:/13960/t3jw96h3m\",\"ppa_cluster_id\":\"aeu.ark:/13960/t3jw96h3m\",\"ppa_work_title\":\"Metra horatiana, or, The metrical systems of Horace\",\"ppa_work_author\":\"Lundy, F. J.\",\"ppa_work_year\":1838,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t5s75qs8g\",\"ppa_source_id\":\"aeu.ark:/13960/t5s75qs8g\",\"ppa_cluster_id\":\"aeu.ark:/13960/t5s75qs8g\",\"ppa_work_title\":\"Pentasyllabic endings in the Latin hexameter with particular reference to the verse of Lucretius\",\"ppa_work_author\":\"Alexander, William Hardy, 1878-1962\",\"ppa_work_year\":1900,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t7qn72r42\",\"ppa_source_id\":\"aeu.ark:/13960/t7qn72r42\",\"ppa_cluster_id\":\"aeu.ark:/13960/t7qn72r42\",\"ppa_work_title\":\"Brief rules of the Latin prosody\",\"ppa_work_author\":\"Cochran, William, 1757-1833\",\"ppa_work_year\":1899,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t8v98vx29\",\"ppa_source_id\":\"aeu.ark:/13960/t8v98vx29\",\"ppa_cluster_id\":\"aeu.ark:/13960/t8v98vx29\",\"ppa_work_title\":\"Introduction to the science of language\",\"ppa_work_author\":\"Sayce, A. H. (Archibald Henry), 1845-1933\",\"ppa_work_year\":1900,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic']\"},{\"ppa_work_id\":\"bc.ark:/13960/t2p63fq11\",\"ppa_source_id\":\"bc.ark:/13960/t2p63fq11\",\"ppa_cluster_id\":\"bc.ark:/13960/t2p63fq11\",\"ppa_work_title\":\"An introduction to Latin elegiac verse composition,\",\"ppa_work_author\":\"Lupton, J. H. (Joseph Hirst), 1836-1905\",\"ppa_work_year\":1913,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"chi.084960397\",\"ppa_source_id\":\"chi.084960397\",\"ppa_cluster_id\":\"chi.084960397\",\"ppa_work_title\":\"A study of the principal Latin rhythms, other than the hexameter ...\",\"ppa_work_author\":\"Clough, Joseph W.\",\"ppa_work_year\":1879,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"}]"' data-total-rows='5539' data-total-columns='8' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["ppa_work_id", ["string", "str"]], ["ppa_source_id", ["string", "str"]], ["ppa_cluster_id", ["string", "st

In [ ]:
import altair as alt

alt.Chart(ppa_ht_df).mark_bar().encode(
    x=alt.X("ppa_work_year", bin=True).axis(format="r"), y="count()"
)

In [ ]:
from zipfile import ZipFile

genre_all_meta_df = pl.read_csv(
    ZipFile("data/1279201/allmeta.csv.zip").read("allmeta.csv"),
    infer_schema_length=10000,
    ignore_errors=True,
)
genre_all_meta_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"htid\":\"chi.086332362\",\"recordid\":8599335,\"oclc\":6618467,\"locnum\":null,\"author\":null,\"imprint\":\"London;Printed by J. Lister;1765.\",\"datetype\":\"s\",\"startdate\":\"1765\",\"enddate\":\" \",\"imprintdate\":\"1765\",\"place\":\"enk\",\"enumcron\":\"c.1\",\"subjects\":\"Europe;Early works to 1800;Agriculture\",\"title\":\"Foreign essays on agriculture and arts\"},{\"htid\":\"chi.086333415\",\"recordid\":9895483,\"oclc\":6312484,\"locnum\":null,\"author\":\"Comber, Thomas,\",\"imprint\":\"London;Printed for W. Nicoll;1772.\",\"datetype\":\"s\",\"startdate\":\"1772\",\"enddate\":\" \",\"imprintdate\":\"1772\",\"place\":\"enk\",\"enumcron\":\"c.1\",\"subjects\":\"England;Agriculture\",\"title\":\"Real improvements in agriculture\"},{\"htid\":\"chi.086333431\",\"recordid\":9895485,\"oclc\":23853418,\"locnum\":null,\"author\":\"Marshall, William\",\"imprint\":\"London;Printed for J. Dodsley;1783.\",\"datetype\":\"s\",\"startdate\":\"1783\",\"enddate\":\" \",\"imprintdate\":\"1783\",\"place\":\"enk\",\"enumcron\":\"c.1\",\"subjects\":\"Early works to 1800;Agriculture\",\"title\":\"Minutes of agriculture;\"},{\"htid\":\"chi.086333449\",\"recordid\":9895486,\"oclc\":7891244,\"locnum\":null,\"author\":\"Peters, Matthew.\",\"imprint\":\"London;Printed for W. Flexney;1771.\",\"datetype\":\"s\",\"startdate\":\"1771\",\"enddate\":\" \",\"imprintdate\":\"1771\",\"place\":\"enk\",\"enumcron\":\"c.1\",\"subjects\":\"Agriculture\",\"title\":\"The rational farmer: or A treatise on agriculture and tillage\"},{\"htid\":\"chi.086333457\",\"recordid\":9895487,\"oclc\":7890994,\"locnum\":null,\"author\":\"Trusler, John,\",\"imprint\":\"London;Printed for the author; and sold by R. Baldwin;1780.\",\"datetype\":\"s\",\"startdate\":\"1780\",\"enddate\":\" \",\"imprintdate\":\"1780\",\"place\":\"enk\",\"enumcron\":\"c.1\",\"subjects\":\"Agriculture\",\"title\":\"Practical husbandry, or, The art of farming, with a certainty of gain: as practised by judicious farmers in this country. The result of experience and long observation\"},{\"htid\":\"chi.095531169\",\"recordid\":9547930,\"oclc\":17448671,\"locnum\":null,\"author\":\"Goldsmith, Oliver,\",\"imprint\":\"London;Printed for F. Wingrave, successor to Mr. Nourse, in the Strand;1791.\",\"datetype\":\"s\",\"startdate\":\"1791\",\"enddate\":\" \",\"imprintdate\":\"1791\",\"place\":\"enk\",\"enumcron\":\"c.1 v.7\",\"subjects\":\"1758-1900;Zoology;Physical geography\",\"title\":\"An history of the earth, and animated nature\"},{\"htid\":\"chi.095531224\",\"recordid\":9547930,\"oclc\":17448671,\"locnum\":null,\"author\":\"Goldsmith, Oliver,\",\"imprint\":\"London;Printed for F. Wingrave, successor to Mr. Nourse, in the Strand;1791.\",\"datetype\":\"s\",\"startdate\":\"1791\",\"enddate\":\" \",\"imprintdate\":\"1791\",\"place\":\"enk\",\"enumcron\":\"c.1 v.8\",\"subjects\":\"1758-1900;Zoology;Physical geography\",\"title\":\"An history of the earth, and animated nature\"},{\"htid\":\"chi.095531575\",\"recordid\":9547930,\"oclc\":17448671,\"locnum\":null,\"author\":\"Goldsmith, Oliver,\",\"imprint\":\"London;Printed for F. Wingrave, successor to Mr. Nourse, in the Strand;1791.\",\"datetype\":\"s\",\"startdate\":\"1791\",\"enddate\":\" \",\"imprintdate\":\"1791\",\"place\":\"enk\",\"enumcron\":\"c.1 v.6\",\"subjects\":\"1758-1900;Zoology;Physical geography\",\"title\":\"An history of the earth, and animated nature\"},{\"htid\":\"chi.096075491\",\"recordid\":9895472,\"oclc\":6536453,\"locnum\":null,\"author\":\"Varlo, Charles,\",\"imprint\":\"London;Printed for the author;1774.\",\"datetype\":\"s\",\"startdate\":\"1774\",\"enddate\":\" \",\"imprintdate\":\"1774\",\"place\":\"enk\",\"enumcron\":\"c.1 v.2\",\"subjects\":\"Early works to 1800;Agriculture\",\"title\":\"A new system of husbandry\"},{\"htid\":\"chi.096075556\",\"recordid\":9895472,\"oclc\":6536453,\"locnum\":null,\"author\":\"Varlo, Charles,\",\"imprint\":\"London;Pri

In [ ]:
ppa_genre_meta = ppa_ht_df.join(
    genre_all_meta_df, left_on="ppa_source_id", right_on="htid", how="inner"
)
ppa_genre_meta

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"ppa_work_id\":\"hvd.hn1ky1\",\"ppa_source_id\":\"hvd.hn1ky1\",\"ppa_cluster_id\":\"greymemoria\",\"ppa_work_title\":\"Memoria technica, or, A new method of artificial memory, :\",\"ppa_work_author\":\"Grey, Richard, 1694-1771\",\"ppa_work_year\":1799,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic', 'Literary']\",\"recordid\":9711520,\"oclc\":19454798,\"locnum\":null,\"author\":\"Grey, Richard,\",\"imprint\":\"London;Printed for W. Lowndes, no.76, Fleet Street; sold by;G. Sael, 192, Strand;T. Hurst, 32, and;J. Wallis, 46, Pater-Noster Row.;1799.\",\"datetype\":\"s\",\"startdate\":\"1799\",\"enddate\":\" \",\"imprintdate\":\"1799\",\"place\":\"enk\",\"enumcron\":null,\"subjects\":\"Mnemonics\",\"title\":\"Memoria technica, or, A new method of artificial memory\"},{\"ppa_work_id\":\"hvd.hn2ghn\",\"ppa_source_id\":\"hvd.hn2ghn\",\"ppa_cluster_id\":\"hvd.hn2ghn\",\"ppa_work_title\":\"Sheridan's and Henderson's practical method of reading and reciting English poetry :\",\"ppa_work_author\":null,\"ppa_work_year\":1796,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic', 'Literary', 'Typographically Unique']\",\"recordid\":9709197,\"oclc\":7879351,\"locnum\":\"PN4105.S63\",\"author\":null,\"imprint\":\"London;E. Newbery, 1796.\",\"datetype\":\"s\",\"startdate\":\"1796\",\"enddate\":\" \",\"imprintdate\":\"1796\",\"place\":\"enk\",\"enumcron\":null,\"subjects\":\"Elocution\",\"title\":\"Sheridan's and Henderson's practical method of reading and reciting English poetry\"},{\"ppa_work_id\":\"hvd.hxjgch\",\"ppa_source_id\":\"hvd.hxjgch\",\"ppa_cluster_id\":\"sheridandiscourse\",\"ppa_work_title\":\"A discourse delivered in the Theatre at Oxford, in the Senate-house at Cambridge, and at Spring-garden in London\",\"ppa_work_author\":\"Sheridan, Thomas, 1719-1788\",\"ppa_work_year\":1759,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\",\"recordid\":9709201,\"oclc\":15522410,\"locnum\":null,\"author\":\"Sheridan, Thomas,\",\"imprint\":\"London;Printed for A. Millar ... J. Rivington and J. Fletcher ... J. Dodsley ... and sold by J. Wilkie ...;1759.\",\"datetype\":\"s\",\"startdate\":\"1759\",\"enddate\":\" \",\"imprintdate\":\"1759\",\"place\":\"enk\",\"enumcron\":null,\"subjects\":\"Elocution\",\"title\":\"A discourse delivered in the Theatre at Oxford, in the Senate-house at Cambridge, and at Spring-garden in London\"},{\"ppa_work_id\":\"hvd.hxkcgn\",\"ppa_source_id\":\"hvd.hxkcgn\",\"ppa_cluster_id\":\"hvd.hxkcgn\",\"ppa_work_title\":\"A dictionary of the English language:\",\"ppa_work_author\":\"Johnson, Samuel, 1709-1784\",\"ppa_work_year\":1773,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Dictionaries', 'Literary']\",\"recordid\":9706179,\"oclc\":5858828,\"locnum\":null,\"author\":\"Johnson, Samuel,\",\"imprint\":\"London;Printed for W. Strahan; J. and F. Rivington; J. Hinton; T. Davies; Hawes, Clarke, and Collins; R. Horsefield; W. Johnston; W. Owen; T. Lowndes; T. Caslon; S. Crowder; T. Longman; B. Law; Beckett, and De Hondt; E. and C. Dilly; J. Dodsley; W. Nicoll; W. Griffin; G. Robinson; T. Cadell; J. Knox; Almon; W. Goldsmith; J. and J. Ridley;1773.\",\"datetype\":\"s\",\"startdate\":\"1773\",\"enddate\":\" \",\"imprintdate\":\"1773\",\"place\":\"enk\",\"enumcron\":null,\"subjects\":\"Dictionaries;English language\",\"title\":\"A dictionary of the English language\"},{\"ppa_work_id\":\"mdp.39015003296970\",\"ppa_source_id\":\"mdp.39015003296970\",\"ppa_cluster_id\":\"monboddoorigin5\",\"ppa_work_title\":\"Of the origin and progress of language.\",\"ppa_work_author\":\"Monboddo, James Burnett, Lord, 1714-1799\",\"ppa_work_year\":1774,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary', 'Original Bibliography']\",\"recordid\":1434875,\"oclc\":1908581,\"locnum\":\"P101.M55\",\"author\":\"Monboddo, James Burnett,\",\"imprint\":\"Edinburgh;J. Balfour;[etc., etc.;1774-92.\",\"datetype\":\"m\",\"sta

In [ ]:
ppa_ht_df.columns

In [ ]:
ppa_genre_meta.write_csv("data/ppa/ppa_genre_metadata_overlap.csv")


mo.md(f"""
- {genre_all_meta_df.height:,} records in page-level genre all-metadata
- {ppa_ht_df.height:,} records in PPA from HathiTrust (excerpts included)
- {ppa_genre_meta.height:,} PPA HathiTrust records matched in page-level genre metadata.
""")

854,476 records in page-level genre all-metadata 
 5,539 records in PPA from HathiTrust (excerpts included) 
 3,536 PPA HathiTrust records matched in page-level genre metadata.

## Test cases

Based on the documented overlap between the datasets, we identified three test cases that would be interesting and helpful to look at.


```
njp.32101068158847-p223 – excerpt didn’t change
Each page has some poetry on it (OG 223-228, digital 267-272)


njp.32101076530979-p482 – excerpt did change
OG page 496 and digital page 504 – quoted poetry
OG page 498 and digital page 506 – quoted poetry
OG page 499 and digital page 507 – big block of quoted poetry
OG page 501 and digital page 509 – quoted poetry
OG page 502 and digital page 510 – quoted poetry
OG page 505 and digital page 513 – quoted poetry
OG page 507 and digital page 515 – ALL POETRY
OG page 508 and digital page 516 – almost all poetry
OG page 509 and digital page 517 – quoted poetry
OG page 510 and digital page 518 – quoted poetry
OG page 511 and digital page 519 – quoted poetry
OG page 512 and digital page 520 – about half quoted poetry

nnc1.0035529865 – not an excerpt, but updated 2023-09-14
Lots of poetry interspersed throughout with prose
Section toward the end with just poems
```

To determine which data file of page-level genre predictions to look at, we need to know when our test cases were published.  It turns out that they are all 1700s, so we can find predictions for all of them in a single data file of the page-level genre dataset.

In [ ]:
# ppa work ids for three test cases of interest
test_cases = [
    "njp.32101068158847-p223",
    "njp.32101076530979-p482",
    "nnc1.0035529865",
]

# page-level genre data is segmented by century, so to determine which data file we need to know the publication years

ppa_test_cases = ppa_ht_df.filter(
    pl.col("ppa_work_id").is_in(test_cases)
).select("ppa_work_id", "ppa_source_id", "ppa_work_title", "ppa_work_year")
ppa_test_cases

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"ppa_work_id\":\"njp.32101068158847-p223\",\"ppa_source_id\":\"njp.32101068158847\",\"ppa_work_title\":\"The Metre of Henry VIII\",\"ppa_work_year\":1758},{\"ppa_work_id\":\"njp.32101076530979-p482\",\"ppa_source_id\":\"njp.32101076530979\",\"ppa_work_title\":\"On Stile and Versification\",\"ppa_work_year\":1799},{\"ppa_work_id\":\"nnc1.0035529865\",\"ppa_source_id\":\"nnc1.0035529865\",\"ppa_work_title\":\"The art of English poetry.\",\"ppa_work_year\":1718}]"' data-total-rows='3' data-total-columns='4' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["ppa_work_id", ["string", "str"]], ["ppa_source_id", ["string", "str"]], ["ppa_work_title", ["string", "str"]], ["ppa_work_year", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
import pathlib
import json

genre_data_dir = pathlib.Path("data/1279201")
all1700_dir = genre_data_dir / "all" / "1700-99"

genre_data_by_work = {}


for row in ppa_test_cases.iter_rows(named=True):
    print(f"\n{row['ppa_work_id']} : {row['ppa_work_title']}")

    # page-level genre data is in a json file named based on the hathi id
    # current test sets don't include any ARKs; if we expand, translate ids to filenames
    with open(all1700_dir / f"{row['ppa_source_id']}.json") as jsonfile:
        genre_data = json.load(jsonfile)

    # display summary information
    genre_counts = genre_data["added_metadata"]["genre_counts"]
    most_common_genre = genre_data["added_metadata"]["maxgenre"]
    print(f"most common genre: {most_common_genre}")
    print("pages by genre:")
    for genre, total in genre_counts.items():
        print(f"\t{genre}: {total}")

    # genre data also includes accuracy scores, could output them also perhaps

    # save the loaded genre data so we can look at pages of interest for our test cases
    genre_data_by_work[row["ppa_work_id"]] = genre_data


njp.32101068158847-p223 : The Metre of Henry VIII
most common genre: non
pages by genre:
	ads: 1
	back: 20
	bio: 23
	dra: 2
	front: 10
	non: 298
	poe: 45

njp.32101076530979-p482 : On Stile and Versification
most common genre: non
pages by genre:
	back: 10
	front: 6
	non: 556
	poe: 2

nnc1.0035529865 : The art of English poetry.
most common genre: poe
pages by genre:
	back: 6
	front: 10
	non: 46
	poe: 278


In [ ]:
# define a utility method so we can easily see sequential ranges of pages in the same genre

from intspan import intspan


def page_genre_chunks(page_genres):
    chunks = []
    current_chunk = []
    current_genre = None

    for page, genre in page_genres.items():
        # special case for the first round
        if current_genre is None:
            current_genre = genre
        # if genre matches, append to current chunk
        if genre == current_genre:
            current_chunk.append(int(page))
        # if genre has changed, save the last chunk and start a new one
        else:
            chunks.append((current_genre, current_chunk))
            current_genre = genre
            current_chunk = [int(page)]

    return chunks


def print_page_chunks(chunks):
    for chunk_genre, page_range in chunks:
        print(f"{chunk_genre}: {intspan(page_range)}")

### Test case 1: njp.32101068158847-p223

This excerpt didn’t change.

Each page has some poetry on it (OG 223-228, digital 267-272)

In [ ]:
# page genre is a dictionary with numeric keys, presumably for the digital page index (starts with zero)
page_genres = genre_data_by_work["njp.32101068158847-p223"]["page_genres"]

# filter to our page range of interest
{
    page: genre
    for page, genre in page_genres.items()
    # if int(page) >= 267 and int(page) <= 272
    if int(page) >= 250 and int(page) <= 280
}
# page_genres

The genre predictions for this are all "non"; even if we expand the page range, all the pages nearby are "non".

There is a preceding section predicted as "bio"; if we could figure out what this is, we might be able to use it to orient ourselves to the changes (but perhaps unhelpful since no poetry predictions).

---

When we look at chunks of sequential pages in the same genre, there's a fair bit of variety.

In [ ]:
print_page_chunks(page_genre_chunks(page_genres))

front: 0-5
non: 6
front: 7-9
non: 10-11
poe: 12-14
front: 15
non: 16-28
poe: 29
non: 30-43
bio: 44-45
non: 46-160
dra: 161-162
non: 163-234
bio: 235-255
non: 256-311
poe: 312-328
non: 329-330
poe: 331-354
back: 355-368
ads: 369


## Test Case 2: njp.32101076530979-p482

This excerpt did change.

- OG page 496 and digital page 504 – quoted poetry
- OG page 498 and digital page 506 – quoted poetry
- OG page 499 and digital page 507 – big block of quoted poetry
- OG page 501 and digital page 509 – quoted poetry
- OG page 502 and digital page 510 – quoted poetry
- OG page 505 and digital page 513 – quoted poetry
- OG page 507 and digital page 515 – ALL POETRY
- OG page 508 and digital page 516 – almost all poetry
- OG page 509 and digital page 517 – quoted poetry
- OG page 510 and digital page 518 – quoted poetry
- OG page 511 and digital page 519 – quoted poetry
- OG page 512 and digital page 520 – about half quoted poetry

In [ ]:
# page genre is a dictionary with numeric keys, presumably for the digital page index (starts with zero)
page_genres2 = genre_data_by_work["njp.32101076530979-p482"]["page_genres"]

# filter to page range of interest
{
    page: genre
    for page, genre in page_genres2.items()
    if int(page) >= 504 and int(page) <= 520
}

When we look at the pages in range for this excerpt, we see only two pages predicted with the genre of poetry (510 and 511).

We suspect that these correspond to the two pages that are all or almost all poetry (515 and 516 in our enumeration), but we're not sure if there's any way to be sure.

In [ ]:
print_page_chunks(page_genre_chunks(page_genres2))

front: 0-5
non: 6-509
poe: 510-511
non: 512-563


When we look at sequences of pages in the same genre, we see that those two pages of poetry were the only content pages flagged with any genre.

### Test Case 3: nnc1.0035529865

This volume is not an excerpt, but one we know was updated 2023-09-14.


- Lots of poetry interspersed throughout with prose
- Section toward the end with just poems

In [ ]:
# page genre is a dictionary with numeric keys, presumably for the digital page index (starts with zero)
page_genres3 = genre_data_by_work["nnc1.0035529865"]["page_genres"]

# this is a long volume with sequences of pages of the same predicted genre
# can we aggregate them into chunks so it's easier to scan?

print_page_chunks(page_genre_chunks(page_genres3))

front: 0-7
non: 8-53
front: 54-55
poe: 56-333


Reporting on pages in the same genre indicates a substantial portion of the volume is predicted to be poetry.